# Preprocess
*Downlaod* data and clean data

In [ ]:
#https://drive.google.com/file/d/1qCVYpb67RuzUbyrJ3w-ohtCf-tzZKTal/view?usp=sharing
#https://drive.google.com/file/d/1dW5SkCYIFbXmNe3xKv4EhrLPDPlXyIDy/view?usp=sharing


!gdown --id 1qCVYpb67RuzUbyrJ3w-ohtCf-tzZKTal
!gdown --id 1dW5SkCYIFbXmNe3xKv4EhrLPDPlXyIDy


import io
import re

def file_clear(in_f, out_f):
    inputFile = io.open(in_f, mode="r", encoding="utf-8") 
    outputFile = io.open(out_f, mode="w", encoding="utf-8") 

    updated=[]
    Lines = inputFile.readlines()
    for i, line in enumerate(Lines):
        newline= re.sub(' +', ' ', line.strip())
        if len(line.split()) >2:    
            updated.append(newline+"\n")
        else:
            print(line,i)    
    outputFile.writelines(updated)
    print( len(Lines), len(updated))
    inputFile.__exit__()
    outputFile.__exit__()
#updated

file_clear('train.txt', 'cleaned_train.txt')
file_clear('test.txt', 'cleaned_test.txt')


Downloading...
From: https://drive.google.com/uc?id=1qCVYpb67RuzUbyrJ3w-ohtCf-tzZKTal
To: /content/train.txt
9.87MB [00:00, 21.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dW5SkCYIFbXmNe3xKv4EhrLPDPlXyIDy
To: /content/test.txt
100% 5.80k/5.80k [00:00<00:00, 12.3MB/s]
بسیارخوری وکامرانی
 2438
هنگام مکافات
 31304
حضرت ستارخان
 57052
سلام چطوری
 67800
گلبنان ممتحن
 74076
همدست وهمداستان
 78314
ای نازنین
 96936
قصر چلستون
 101252
بسیار سخن
 107340
اندر آذربایجان
 108670
نپذیرند یکی
 118562
بیشتر کن
 140401
بیچاره         رعیت
 151833
اندرگلستان ما
 152975
ناگاه صبا
 159903
پرشرر کن
 167759
همدرد ندارد
 175501
برملت خوبشتن
 184457
188894 188876
108 108


# Model Codes

In [ ]:
from itertools import product
import math
import nltk
from pathlib import Path

In [ ]:
import nltk

SOS = "<s> "
EOS = "</s>"
UNK = "<UNK>"

def add_sentence_tokens(sentences, n):
    """Wrap each sentence in SOS and EOS tokens.
    For n >= 2, n-1 SOS tokens are added, otherwise only one is added.
    Args:
        sentences (list of str): the sentences to wrap.
        n (int): order of the n-gram model which will use these sentences.
    Returns:
        List of sentences with SOS and EOS tokens wrapped around them.
    """
    sos = SOS * (n-1) if n > 1 else SOS
    return ['{}{} {}'.format(sos, s, EOS) for s in sentences]

def replace_singletons(tokens):
    """Replace tokens which appear only once in the corpus with <UNK>.
    
    Args:
        tokens (list of str): the tokens comprising the corpus.
    Returns:
        The same list of tokens with each singleton replaced by <UNK>.
    
    """
    vocab = nltk.FreqDist(tokens)
    return [token if vocab[token] > 1 else UNK for token in tokens]

def preprocess(sentences, n):
    """Add SOS/EOS/UNK tokens to given sentences and tokenize.
    Args:
        sentences (list of str): the sentences to preprocess.
        n (int): order of the n-gram model which will use these sentences.
    Returns:
        The preprocessed sentences, tokenized by words.
    """
    sentences = add_sentence_tokens(sentences, n)
    tokens = ' '.join(sentences).split(' ')
    tokens = replace_singletons(tokens)
    return tokens


#from preprocess import preprocess


def load_data(train_path, test_path=""):
    """Load train and test corpora from a directory.
    Directory must contain two files: train.txt and test.txt.
    Newlines will be stripped out. 
    Args:
        data_dir (Path) -- pathlib.Path of the directory to use. 
    Returns:
        The train and test sets, as lists of sentences.
    """
    # train_path = data_dir.joinpath('train.txt').absolute().as_posix()
    # test_path  = data_dir.joinpath('test.txt').absolute().as_posix()

    with open(train_path, 'r') as f:
        train = [l.strip() for l in f.readlines()]

    if test_path != "":  
        with open(test_path, 'r') as f:
            test = [l.strip() for l in f.readlines()]
    else:
        return train        
    return train, test

In [ ]:
class LanguageModel(object):
    """An n-gram language model trained on a given corpus.
    
    For a given n and given training corpus, constructs an n-gram language
    model for the corpus by:
    1. preprocessing the corpus (adding SOS/EOS/UNK tokens)
    2. calculating (smoothed) probabilities for each n-gram
    Also contains methods for calculating the perplexity of the model
    against another corpus, and for generating sentences.
    Args:
        train_data (list of str): list of sentences comprising the training corpus.
        n (int): the order of language model to build (i.e. 1 for unigram, 2 for bigram, etc.).
        laplace (int): lambda multiplier to use for laplace smoothing (default 1 for add-1 smoothing).
    """

    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        """Apply Laplace smoothing to n-gram frequency distribution.
        
        Here, n_grams refers to the n-grams of the tokens in the training corpus,
        while m_grams refers to the first (n-1) tokens of each n-gram.
        Returns:
            dict: Mapping of each n-gram (tuple of str) to its Laplace-smoothed 
            probability (float).
        """
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        """Create a probability distribution for the vocabulary of the training corpus.
        
        If building a unigram model, the probabilities are simple relative frequencies
        of each token with the entire corpus.
        Otherwise, the probabilities are Laplace-smoothed relative frequencies.
        Returns:
            A dict mapping each n-gram (tuple of str) to its probability (float).
        """
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        """Convert, if necessary, a given n-gram to one which is known by the model.
        Starting with the unmodified ngram, check each possible permutation of the n-gram
        with each index of the n-gram containing either the original token or <UNK>. Stop
        when the model contains an entry for that permutation.
        This is achieved by creating a 'bitmask' for the n-gram tuple, and swapping out
        each flagged token for <UNK>. Thus, in the worst case, this function checks 2^n
        possible n-grams before returning.
        Returns:
            The n-gram with <UNK> tokens in certain positions such that the model
            contains an entry for it.
        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        """Calculate the perplexity of the model against a given test corpus.
        
        Args:
            test_data (list of str): sentences comprising the training corpus.
        Returns:
            The perplexity of the model as a float.
        
        """
        test_tokens = preprocess(test_data, self.n)
        test_ngrams = nltk.ngrams(test_tokens, self.n)
        N = len(test_tokens)

        known_ngrams  = (self._convert_oov(ngram) for ngram in test_ngrams)
        probabilities = [self.model[ngram] for ngram in known_ngrams]
        
        return math.exp((-1/N) * sum(map(math.log, probabilities)))

    def _best_candidate(self, prev, i, without=[]):
        """Choose the most likely next token given the previous (n-1) tokens.
        If selecting the first word of the sentence (after the SOS tokens),
        the i'th best candidate will be selected, to create variety.
        If no candidates are found, the EOS token is returned with probability 1.
        Args:
            prev (tuple of str): the previous n-1 tokens of the sentence.
            i (int): which candidate to select if not the most probable one.
            without (list of str): tokens to exclude from the candidates list.
        Returns:
            A tuple with the next most probable token and its corresponding probability.
        """
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</s>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<s>" else i]
     
    def generate_sentences(self, num, min_len=12, max_len=24):
        """Generate num random sentences using the language model.
        Sentences always begin with the SOS token and end with the EOS token.
        While unigram model sentences will only exclude the UNK token, n>1 models
        will also exclude all other words already in the sentence.
        Args:
            num (int): the number of sentences to generate.
            min_len (int): minimum allowed sentence length.
            max_len (int): maximum allowed sentence length.
        Yields:
            A tuple with the generated sentence and the combined probability
            (in log-space) of all of its n-grams.
        """
        for i in range(num):
            sent, prob = ["<s>"] * max(1, self.n-1), 1
            while sent[-1] != "</s>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</s>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob
                
                if len(sent) >= max_len:
                    sent.append("</s>")

            yield ' '.join(sent), -1/math.log(prob)

    def complete_sentence(self, phrase, missing_count): #min_len=12, max_len=24):
        sent, prob = ["<s>"] * max(1, self.n-1), 1
        sent = sent + [ x.strip() for x in phrase.split() if len(x)>1]
        for i in range(missing_count):
            prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
            blacklist = sent + ["</s>"] #(["</s>"] if len(sent) < min_len else [])
            next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
            sent.append(next_token)
            prob *= next_prob
            
            # if len(sent) >= max_len:
            #     sent.append("</s>")
        sent = sent + ["</s>"] * max(1, self.n-1)
        return ' '.join(sent), -1/math.log(prob)       

In [ ]:
class Args:
    def __init__(self, train_f="cleaned_train.txt", test_f="cleaned_test.txt",  n=2, laplace=.01, num=3):
        self.train_f = train_f
        self.test_f = test_f

        self.n=n
        self.laplace=laplace
        self.num=num
args = Args()


# Load and prepare train/test data
# data_path = Path(args.data)
# train, test = load_data(data_path)
train, test = load_data(Path(args.train_f),Path(args.test_f))

print("Loading {}-gram model...".format(args.n))
lm = LanguageModel(train, args.n, laplace=args.laplace)
print("Vocabulary size: {}".format(len(lm.vocab)))

print("Generating sentences...")
for sentence, prob in lm.generate_sentences(args.num):
    print("{} ({:.5f})".format(sentence, prob))

# perplexity = lm.perplexity(test)
# print("Model perplexity: {:.3f}".format(perplexity))
# print("")

Loading 2-gram model...
Vocabulary size: 27927
Generating sentences...
<s> که در آن را به دست و از تو بر سر </s> (0.02418)
<s> به دست و از آن را ز تو در دل من </s> (0.02418)
<s> از آن را به دست و در دل من ز تو </s> (0.02405)


# Q1 Dirichlet Prior

In [ ]:
sents_to_comp=[ ("چون مشک سیه بود مرا هر دو", 2),
                ("گر خورد سوگند هم آن", 1),
                ("زانک نفس آشفته تر گردد از",1),
                ("ازین زشت تر در جهان رنگ",1),
                ("دوست در خانه و ما گرد",2),
                ("شب است و شمع و شراب و",1)
]

for phrase, missing_count in sents_to_comp:
    completed = lm.complete_sentence(phrase, missing_count)
    print(completed)


('<s> چون مشک سیه بود مرا هر دو چشم من </s>', 0.17603589641442205)
('<s> گر خورد سوگند هم آن را </s>', 0.26795928100828315)
('<s> زانک نفس آشفته تر گردد از آن </s>', 0.3208489208062978)
('<s> ازین زشت تر در جهان رنگ و </s>', 0.45793907493459973)
('<s> دوست در خانه ما گرد و از </s>', 0.12235655018702388)
('<s> شب است شمع شراب و </s>', 0.36522037121905526)


# Q2 Perpelexity

In [ ]:
lm.perplexity(test)
#lm.model

40.69355828796971

#Q3 Neural Language Model

In [ ]:
#Remove Big Prev Model 
lm = None

#Get Words
from nltk import FreqDist
from nltk.util import ngrams    
import collections

def compute_freq(inputfile, nu):
    textfile = io.open(inputfile, mode="r", encoding="utf-8")
    Lines = textfile.readlines()
    textfile.__exit__()            
    ngramfdist = FreqDist()
 
    for i,line in enumerate(Lines):
        tokens = line.strip().split(' ')
        if len(tokens)>nu:
            # ngram_ = ngrams(tokens, nu)
            # print(i+1," ",line)
            ngram_ = ngrams(tokens, nu)
            ngramfdist.update(ngram_)
    return ngramfdist

words = compute_freq("cleaned_train.txt",1)
#words_dict = collections.Counter(ngrams)
words = [i[0] for i in words]
word2int = dict(enumerate(words))
word2int = {word: ind for ind, word in word2int.items()}
int2word = {value: key for key, value in word2int.items()}
int2word.keys()

vocab_size= len(int2word)

#Get Trigrams
def generateSeq(inputfile, nu):
    textfile = io.open(inputfile, mode="r", encoding="utf-8")
    Lines = textfile.readlines()
    textfile.__exit__()            
    sequences={}
    seqIndex=0
    for i,line in enumerate(Lines):
        tokens = line.strip().split(' ')
        if len(tokens)>nu:
            ngrams_ = ngrams(tokens, nu)
            for tuple_ in ngrams_:
                t = list(tuple_)
                sequences[seqIndex]= ([word2int[i] for i in t[:-1]], word2int[t[-1]] )    
                seqIndex = seqIndex+1
    return sequences

sequences_dict = generateSeq("cleaned_train.txt",3)


In [ ]:
#input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
import torch 
import torch.nn as nn
import torch.nn.functional as F

CONTEXT_SIZE = 2
EMBEDDING_DIM = 128

import torch

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, sequence_dict):
        'Initialization'
        self.data_dict = sequence_dict
        # self.labels = labels
        # self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_dict)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        # ID = self.list_IDs[index]

        # # Load data and get label
        # X = torch.load('data/' + ID + '.pt')
        # y = self.labels[ID]
        x,y = self.data_dict[index]
        return tuple(x), y

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        #embeds = self.embeddings(inputs).view((1, -1))
        embeds= torch.cat((self.embeddings(inputs[0]),self.embeddings(inputs[1])),1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# Parameters
params = {'batch_size': 1024*8,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 100

# Generators
training_set = Dataset(sequences_dict)
training_generator = torch.utils.data.DataLoader(training_set, **params)


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(max_epochs):
    total_loss = 0
    for context, target in training_generator:

        context_idxs = [context[0].to(device), context[1].to(device)] 
        model.zero_grad()

        log_probs = model(context_idxs)

        loss = loss_function(log_probs, target.to(device))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(epoch, " ",total_loss)
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"


972.4722065925598
835.3044457435608
811.8480472564697
792.6217665672302
773.9694743156433
754.6817936897278
734.3321185112
713.6036310195923
694.2062292098999
677.9491186141968
665.4597520828247
655.7763133049011
647.8947401046753
641.1842975616455
635.1420650482178
629.7935228347778
624.8499727249146
620.3488807678223
616.0712938308716
612.0496368408203
608.3208708763123
604.7394375801086
601.3440775871277
598.1088042259216
594.9670538902283
591.9797043800354
589.0549421310425
586.2892870903015
583.5651874542236
580.9411664009094
578.401029586792
575.863477230072
573.5538420677185
571.1129097938538
568.8731560707092
566.5863628387451
564.4155249595642
562.2446885108948
560.1706924438477
558.1298184394836
556.1356468200684
554.2393755912781
552.268792629242
550.3745546340942
548.5836338996887
546.7639651298523
544.9467716217041
543.1820521354675
541.508894443512
539.8328895568848
538.2073760032654
536.6298027038574
535.0380930900574
533.4468946456909
531.9985151290894
530.4819054603577

In [ ]:
import numpy as np
sents_to_comp=[ ("چون مشک سیه بود مرا هر دو", 2),
                ("گر خورد سوگند هم آن", 1),
                ("زانک نفس آشفته تر گردد از",1),
                ("ازین زشت تر در جهان رنگ",1),
                ("دوست در خانه و ما گرد",2),
                ("شب است و شمع و شراب و",1)
]

model.eval()
for sent, i2 in  sents_to_comp:
    for i in range(i2):
        sent_words= sent.strip().split()
        context = [ torch.tensor([word2int[sent_words[-2]]]).to(device) , torch.tensor([word2int[sent_words[-1]]]).to(device)]
        nextWordPreds = model(context)
        nextWordID = np.argmax(nextWordPreds.detach().cpu()).item()
        nextWord = int2word[nextWordID]
        sent = sent + " "+ nextWord
    print(sent)




چون مشک سیه بود مرا هر دو را به
گر خورد سوگند هم آن را
زانک نفس آشفته تر گردد از اجلال
ازین زشت تر در جهان رنگ و
دوست در خانه و ما گرد تو بر
شب است و شمع و شراب و طعام
